In [1]:
#   ___                  _                
#  / _/______ ____  ____(_)__ _______     
# / _/ __/ _ `/ _ \/ __/ (_-</ __/ _ \    
#/_//_/  \_,_/_//_/\__/_/___/\__/\___/    
# ___ _____(_)__ ___ ____  / /_(_)       
# / _ `/ __/ (_-</ _ `/ _ \/ __/ /        
# \_, /_/ /_/___/\_,_/_//_/\__/_/         
#/___/
#
#Samee Lab @ Baylor College Of Medicine
#francisco.grisanticanozo@bcm.edu

In [9]:
library("Seurat")
library("scPred")
library("tidyverse")
require("SingleCellExperiment")
library("doParallel")


# set up future for parallelization
library(future)
library(future.apply)
plan("multiprocess", workers = 10)
options(future.globals.maxSize = 20000 * 1024^2)


Attaching package: ‘future’


The following object is masked from ‘package:SummarizedExperiment’:

    values


The following object is masked from ‘package:GenomicRanges’:

    values


The following object is masked from ‘package:IRanges’:

    values


The following object is masked from ‘package:S4Vectors’:

    values




# Load data

In [8]:
adata_train <- readRDS('../Data/Tabula_Sapiens/adata_train.rds')
adata_predict <- readRDS('../Data/Tabula_Sapiens/adata_predict.rds')

In [11]:
adata_train <- adata_train  %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA() 

Centering and scaling data matrix

PC_ 1 
Positive:  SERPINB4, CALML3, CHP2, PAX9, H19, SERPINB11, DSC3, AC025154.2, TNNT3, LINC01133 
	   SLC16A9, SMIM31, ERN2, NTN1, KRT4, EYA1, CKMT1A, SCARA3, GJB5, EYA4 
	   ALDH1L1, LY6D, DMRT3, SOX21-AS1, NCS1, WNK2, MDFI, RARB, GBP6, FRMD6 
Negative:  CD38, RORB, AMPH, FGL1, LINC01736, FHOD1, STC1, FAM19A5, DPY19L1, SLC6A3 
	   ADAMTS7P4, AC002059.1, KRT16P2, CSGALNACT1, AL138900.2, AL160408.1, SCIN, SLC4A7, AC243829.1, CST2 
	   TIMP4, FRG1FP, NECTIN3, AC007687.1, AC078850.1, FTLP17, DRGX, MTAPP2, NTSR1, MYT1 
PC_ 2 
Positive:  CALML3, CHP2, SERPINB4, SLC16A9, SERPINB11, ERN2, H19, SOX21-AS1, VAV3, LY6D 
	   AC025154.2, CKMT1A, GBP6, WNK2, TNNT3, AC084816.1, SMIM31, AMPH, TENM4, LINC01736 
	   EYA4, DMRT3, SLC4A7, TMPRSS11D, B4GALNT2, HSD11B2, LINC01133, CSGALNACT1, HLA-V, TIMP4 
Negative:  ATOH8, DLL1, SMOC2, SOX9, COL17A1, PTPRR, NCS1, IFT81, PPP1R13B, HMCN1 
	   FBXO17, MIXL1, RORC, ST6GAL2, KL, FKBP1B, SMO, RNF165, NECTIN3, SCARA3 
	   GHR,

In [12]:
adata_predict <- adata_predict %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA() 

Centering and scaling data matrix

PC_ 1 
Positive:  MUC5B, FAM155A-IT1, CTSK, LINC02608, MIR3188, MMP12, DDX43, ANO3, HABP2, UCN3 
	   RPL7P54, AL159152.1, RPL23AP2, SLC7A11-AS1, FRG1JP, FTH1P2, AC004233.2, XAGE3, EGFLAM, HSPA7 
	   TMEM155, LGALS7, B4GALT3, GAPDHP66, CSMD2, ZAR1, MIR9-3HG, RN7SKP137, SNORD3C, AC025884.1 
Negative:  IL1A, AC022509.2, KRT79, PNPLA3, WFIKKN2, AL136097.2, CES1P1, AC067752.1, SHKBP1, PCARE 
	   DUSP27, GMPR, AL772337.2, AC022509.4, IDNK, GAPDHP14, AC091305.1, OR10AA1P, AP000688.2, CEACAM8 
	   LINC02201, GBP7, WNK4, EEF1AKNMT, A4GNT, INHBA-AS1, GMPPA, PI4KAP1, FAM9B, MBD1 
PC_ 2 
Positive:  MUC5B, IL1A, B4GALT3, AC012653.2, CEACAM8, PCARE, AC091305.1, MTCO2P22, AL772337.2, DUSP27 
	   FAM155A-IT1, KRT79, LINC00656, OPRK1, MBD1, AC022509.2, WNK4, INHBA-AS1, PNPLA3, AC022509.4 
	   TMPRSS11E, LINC02608, RN7SL795P, MIR3188, CES1P1, AP000688.2, CTRC, PI4KAP1, HABP2, AC246817.2 
Negative:  CTSK, AC096537.1, MANSC4, MMP12, GMPR, TM4SF19-TCTEX1D2, NMRK2, AL35561

# Run integration pipeline

In [13]:
anchors <- FindTransferAnchors(reference = adata_train, 
                               query = adata_predict, 
                               dims = 1:30)

Performing PCA on the provided reference using 2000 features as input.

Projecting PCA

Finding neighborhoods

Finding anchors

	Found 8768 anchors

Filtering anchors

	Retained 6700 anchors



In [14]:
predictions <- TransferData(anchorset = anchors, 
                            refdata = adata_train$celltype,
                            dims = 1:30)

Finding integration vectors

Finding integration vector weights

Predicting cell labels



In [15]:
adata_predict <- AddMetaData(adata_predict, metadata = predictions)

# Save results

In [17]:
write.csv(adata_predict@meta.data,'../outputs/Tabula_Sapiens_predictions_Seurat.csv')